# Appendix A: Photometric Errors

In doing photometry, you may have encountered situation what the errors actually mean. In this appendix, I will briefly explain the magnitude errors and its derivation.

## 1. Pixel-Wise Noise
CCD itself has two sources of errors which is everywhere on CCD pixels. They are the **Readout noise** and the **Poisson noise**. Readout noise is a Gaussian noise, which *must* be given by the CCD manufacturer in *electron unit*. It is caused since a CCD is not perfect when counting the electrons inside the potential well. So even if we have 10 electrons, the CCD may give us that it got 7, 13, 9, ... electrons from time to time. The readout noise is the standard deviation of such imperfect counting from the CCD.

The Poisson noise occurs because what we get is the electron count. Due to the photon input, CCD pixel generates photoelectrons. This process is a Poissonian process, and thus it has inevitable Poisson noise. If the electron count is $N$, the Poisson noise, i.e., the standard deviation, is $\sqrt{N}$. Note that this $N$ is in electron unit, not in ADU, so you must multiply the effective gain factor to the ADU counts.

These two noises are combined by error propagation (ignoring covariance as they are independent) as $\sqrt{N + R^2} = \sqrt{gN_{\rm ADU} + R^2}$ in electrons. $g$ means the effective gain and the subscript ADU means the unit of it is in ADU. Keep in mind that this error should **always** be considered for **every** pixel.

-----


## 2. Sky Value Estimation

We take an aperture to sum the total pixel values within the aperture and set an annulus to estimate the sky. But how do we estimate the best sky value and its uncertainty? 

### Mathematical Conclusion

First, in mathematical sense, the conclusion is that the true mean value of sky has expectation value of the sample mean of the sky pixels, and the uncertainty of it is calculated by Student-t distribution. In mathematical term, it is written as 

\begin{align}
	\frac{m_s - \mu_s}{s_s/\sqrt{n_s}} &\sim T_{n_s-1} \\
    \mu_s &= m_s \pm t^{\alpha}_{n_s-1} \frac{s_s}{\sqrt{n_s}}
\end{align}

$m_s$ and $\mu_s$ are the sample mean and true mean of sky, $s_s$ is the sample standard deviation of sky, $n_s$ is the number of sky pixels used for calculating $m_s$ and $s_s$, and $t^{\alpha}_{n_s-1}$ is the $t$ value above which the cdf of T-distribution with dof $n_s-1$ is $\alpha$. The $t$-value table is calculable by softwares, and some critical values are given at many web sites such as [here](http://math.tutorvista.com/statistics/t-distribution-table.html). If $n_s \rightarrow \infty$ (practically if larger than roughly about 30), the t-distribution is well approximated by Gaussian (normal) distribution.

### Astronomers' Language

But astronomers, especially if we are not interested in very accurate measurement, we only want to use simpler and informal formalism. In the following, I will explain the astronomers' language by introducing some mathematical theorems to justify it.

Consider you set an annulus with inner and outer radius of 10 and 20 pixels. Then there may be hundreds of pixels within the annulus, and they will be sigma clipped so that unwanted sources, bad pixels, etc, will be rejected. Say we are left with $n_s$ pixels. 

The frequency plot (histogram) of sky values will be somewhat irregular in its shape some times. But in mathematical sense, we have a good theorem called the **central limit theorem (CLT)**:

> Say [i.i.d.](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables) random variable $X \sim G(\mu, \sigma)$ ($X$ follows an unknown general probability distribution with true mean $\mu$ and true stdev $\sigma$). If we draw $n (\gg 1)$ samples, the sample mean will represent the true mean. More rigorously, the sample mean ($\bar{X}$) will approximately follow a Gaussian distribution:   
\begin{equation}
	\bar{X} \dot{\sim} \mathcal{N}(\mu, ( \sigma/\sqrt{n})^2)
\end{equation}

In our sky estimation, this means that if we take the sample mean of the pixel values from the sky annulus ($m_s$), it will be similar to the true sky mean ($\mu_s$), with uncertainty of true sky stdev divided by the sqrt of sky pixels ($\sigma_s/\sqrt{n_s}$):

\begin{equation}
	\mu_s \approx m_s \pm \frac{\sigma_s}{\sqrt{n_s}}
\end{equation}

This is true only when we have many sky pixels ($n_s \gg 1$). But how do we know the *true* sky stdev? We have another useful theorem which says we can assume $\sigma_s \approx s_s$, where $s_s$ is the *sample* sky stdev:

> For the same situation as CLT, the sample stdev ($ s^2 := \sum_i (X_i - \bar{X})^2 / (n-1) $) and the true stdev has following property:
\begin{equation}
	\frac{(n-1) s^2}{\sigma^2} \sim \chi^2_{n-1}
\end{equation}
In other words, $(n-1) s^2/\sigma^2$ follows a chi-square distribution with dof $(n-1)$. From the definition of chi-square distribution, one can calculate that the expected value of $s$ is $\sigma$. 

So we have

\begin{equation}
	\mu_s \approx m_s \pm \frac{s_s}{\sqrt{n_s}}
\end{equation}

It resembles the mathematical conclusion! This is when the t-value is 1, i.e., 1-sigma uncertainty.



### Why Mean?

Although all the logic we constructed above is based on rigorous mathematical theorems, mean is not always a good choice in practice. For example, the sigma-clipping with respect to the mean may cause severe problem if we have a few cosmic-rays (why?), since cosmic rays are un-wanted external interference, which is not from the "sky". In general, therefore, the sigma-clipping itself is done around median value, and thus the sky estimation is closely related to the median as well as to the mean.

So [SExtractor](http://www.astromatic.net/software/sextractor) uses its own background estimation method, and some more accurate description is given in [photutils document](https://photutils.readthedocs.io/en/stable/photutils/background.html#d-background-and-noise-estimation). SExtractor uses `(2.5 * median) - (1.5 * mean)`, not just `mean`, as the $m_s$. Only when `(mean - median) / std > 0.3` the `median` is used. IDL uses so-called MMM method, and IRAF has its own method, which are similar to SExtractor but only some of the coefficients are different. All these are empirical estimator of the *mode*, not the mean or median (a function of these two to estimate the mode).

I haven't yet read/find rigorous reasoning for these, but SExtractor User's Manual says *mode is more stable than mean* when we are looking at a crowded ("many-star") region. This is intuitively reasonable, because the mean may be affected by stellar flux even after sigma clipping, when there are too many nearby stars in the annulus. The mode, however, is a bit noisier (uncertain) than mean, which also makes sense thinking about the nearby stars' flux within annulus, so it is better to use mean when we have few stars in the image. The the above reasoning using mean makes sense in more rigorous manner.


### Summary

Although we need to use the Student-t distribution to be rigorous, astronomers are satisfied with only the 1-sigma range, i.e., 

\begin{equation}
	\mu_s \approx m_s \pm \frac{s_s}{\sqrt{n_s}}~.
\end{equation}

This will suffice many of the astronomers without the explanation of t-distribution. This is because we have sky pixels of hundreds, and the t-distribution will be approximately Gaussian, so the uncertainty term in the equation can be roughly understood as the stdev of Gaussian distribution. 

Depending on softwares, different mode estimation methods are used to infer $\mu_s$. Mode is especially useful for crowded field photometry.

In the above estimations, however, we did not explicitly use the pixel-wise noise terms. As a rough estimate, we can add these error terms by error propagation: 

\begin{equation}
	\mu_s \approx m_s \pm \frac{s_s}{\sqrt{n_s}} \pm \sqrt{m_s + n_s R^2}~.
\end{equation}

-----

## 3. Stellar Flux Estimation

In aperture photometry, we estimate the stellar flux as `sum - n_a * m_s`, i.e., the sum of all pixel values in the aperture, and subtract the sky counts per pixel times the number of pixels in aperture ($n_a$). Since the `sum` is easy to calculate, `n_a` is what we can change, and `m_s` is obtained from previous section. Then what is the uncertainty of this stellar flux?


